In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from art.utils import load_mnist
from art.estimators.classification.pytorch import PyTorchClassifier
from art.metrics.metrics import clever_t, clever_u, clever

# Create the classifier

In [2]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv = torch.nn.Conv2d(1, 16, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.fc = torch.nn.Linear(2304, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x = x.view(-1, 2304)
        logit_output = self.fc(x)

        return logit_output

# Define the network
model = Model()

# Define a loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Get classifier
ptc = PyTorchClassifier(
    model=model, loss=loss_fn, optimizer=optimizer, input_shape=(1, 28, 28), nb_classes=10, clip_values=(0, 1)
)

In [3]:
ptc

/Users/aromanov/code/microsoft/ai_defender/adversarial_robustness_toolbox/art/estimators/classification/pytorch.py:721: DeprecationWarning: Function 'channel_index' is deprecated and will be removed in future release 1.6.0. It will be replaced by 'channels_first'.
  self.channel_index,


art.estimators.classification.pytorch.PyTorchClassifier(model=ModelWrapper(
  (_model): Model(
    (conv): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc): Linear(in_features=2304, out_features=10, bias=True)
  )
), loss=CrossEntropyLoss(), optimizer=Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
), input_shape=(1, 28, 28), nb_classes=10, channel_index=<art.utils._Deprecated object at 0x7faa909d9050>, channels_first=True, clip_values=array([0., 1.], dtype=float32), preprocessing_defences=None, postprocessing_defences=None, preprocessing=[StandardisationMeanStdPyTorch(mean=0, std=1, apply_fit=True, apply_predict=True, device=cpu)])

# Load data

In [4]:
# Get MNIST
batch_size, nb_train, nb_test = 100, 1000, 10
(x_train, y_train), (x_test, y_test), _, _ = load_mnist()
x_train, y_train = x_train[:nb_train], y_train[:nb_train]
x_test, y_test = x_test[:nb_test], y_test[:nb_test]
x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

/Users/aromanov/code/microsoft/ai_defender/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
x_train.shape, y_train.shape

((1000, 1, 28, 28), (1000, 10))

In [6]:
x_test.shape, y_test.shape

((10, 1, 28, 28), (10, 10))

# Train the model

In [7]:
ptc.fit(x_train, y_train, batch_size=batch_size, nb_epochs=3)

# Calc CLEVER

In [8]:
R_L1 = 40
R_L2 = 2
R_LI = 0.1

In [11]:
# Test targeted clever
res0 = clever_t(ptc, x_test[-1], 2, 10, 5, R_L1, norm=1, pool_factor=3)
res1 = clever_t(ptc, x_test[-1], 2, 10, 5, R_L2, norm=2, pool_factor=3)
res2 = clever_t(ptc, x_test[-1], 2, 10, 5, R_LI, norm=np.inf, pool_factor=3)
print(f'Targeted: {res0:.3f}, {res1:.3f}, {res2:.3f}')

Targeted: 12.762, 2.000, 0.100


In [12]:
# Test untargeted clever
res0 = clever_u(ptc, x_test[-1], 10, 5, R_L1, norm=1, pool_factor=3)
res1 = clever_u(ptc, x_test[-1], 10, 5, R_L2, norm=2, pool_factor=3)
res2 = clever_u(ptc, x_test[-1], 10, 5, R_LI, norm=np.inf, pool_factor=3)
print(f'Untargeted: {res0:.3f}, {res1:.3f}, {res2:.3f}')


Untargeted: 0.512, 0.102, 0.006
